In [1]:
# import libs
import numpy as np
import pandas as pd
import os
import scipy.io as io
from sklearn import preprocessing
from scipy.stats import pearsonr
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from numpy import array
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
import math
from scipy.linalg import expm,logm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

In [2]:
## 计算黎曼距离
def riemann_distance(X,Y):
    X = np.reshape(X,(26,26))
    Y = np.reshape(Y,(26,26))
    gtype = 'A'
#     print(X.shape)
    if gtype == 'A':
        tmpEig = geneig(X,Y)
        tmp_dis = sum(mylog(tmpEig) ** 2)
    elif gtype == 'S':
        t = mylog(np.linalg.det(0.5 * (X + Y))) - 0.5 * (mylog(np.linalg.det(X))) + mylog(np.linalg.det(Y))
        if t == np.inf or np.isnan(t):
            eigX = np.linalg.eig(X)
            eigX = np.maximum(eigX,eps)
            eigY = np.linalg.eig(Y);
            eigY = np.maximum(eigY,eps)

            t = np.real(sum(mylog(np.linalg.eig(0.5*(X+Y))))) - 0.5 * (np.real(sum(mylog(eigX)))) + np.real(sum(mylog(eigY)))

        tmp_dis = t;
    elif gtype == 'J':
        tmp_dis = 0.5 * (np.dot(np.linalg.inv(Y),X)).trace() + 0.5 * (np.dot(np.linalg.inv(X),Y)).trace() - X.shape[0]
    elif gtype == 'L':
        tmp_dis = np.linalg.norm((logm(X) - logm(Y)),ord='fro') ** 2
    
    if tmp_dis <= 1e-15:
        dis = 0
    else:
        dis = np.sqrt(tmp_dis);
    return dis

## 计算广义特征值
def geneig(X,Y):
    w,v = np.linalg.eig(np.dot(np.linalg.inv(Y),X))
    return w

## 计算可能出现负数的对数函数
def mylog(X):
    CX = X + 0j
    return np.log(CX)

## 将协方差矩阵转成上三角矩阵
def dealSPD(A):
    n = A.shape[0]
    B = np.zeros(shape=(1, n*(n+1)//2))
    j = 0
    for i in range(0,n):
        B[:,j:j+n-i] = A[i,i:n]
        j = j + n -i
    return B


In [3]:
riskfile = '/home/ubuntu/xuhao/data/2019riskdata2.csv'
riskdata = pd.read_csv(riskfile)

In [4]:
## 20features
ExtraTreeFeatures = [' Total Fwd Packets', 'Total Length of Fwd Packets', ' Fwd Packet Length Max', ' Fwd Packet Length Min', ' Fwd Packet Length Mean', 'Flow Bytes/s', ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Max', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Max', 'Fwd Packets/s', ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean', ' ACK Flag Count', ' Average Packet Size', ' Avg Fwd Segment Size', ' Subflow Fwd Bytes']

LogisticRegressionFeatures =['Total Length of Fwd Packets', ' Fwd Packet Length Min', ' Fwd Packet Length Mean', ' Fwd Packet Length Std', ' Bwd Packet Length Min', 'Flow Bytes/s', ' Flow Packets/s', ' Flow IAT Mean', 'Fwd IAT Total', ' Fwd IAT Std', 'Fwd Packets/s', ' Min Packet Length', ' Packet Length Mean', ' Packet Length Std', ' Packet Length Variance', ' ACK Flag Count', ' Average Packet Size', ' Avg Fwd Segment Size', 'Subflow Fwd Packets', ' Subflow Fwd Bytes']
# ## 30 features
# ExtraTreeFeatures=[' Flow Duration', ' Total Fwd Packets', 'Total Length of Fwd Packets', ' Fwd Packet Length Max', ' Fwd Packet Length Min', ' Fwd Packet Length Mean', ' Fwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max', ' Fwd IAT Min', 'Fwd Packets/s', ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean', ' Packet Length Std', ' PSH Flag Count', ' ACK Flag Count', ' URG Flag Count', ' Average Packet Size', ' Avg Fwd Segment Size', 'Subflow Fwd Packets', ' Subflow Fwd Bytes']
# LogisticRegressionFeatures=[' Flow Duration', ' Total Fwd Packets', ' Total Backward Packets', 'Total Length of Fwd Packets', ' Fwd Packet Length Max', ' Fwd Packet Length Min', ' Fwd Packet Length Mean', ' Fwd Packet Length Std', ' Bwd Packet Length Min', 'Flow Bytes/s', ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', 'Fwd IAT Total', ' Fwd IAT Std', ' Bwd Header Length', 'Fwd Packets/s', ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean', ' Packet Length Std', ' Packet Length Variance', ' PSH Flag Count', ' ACK Flag Count', ' Average Packet Size', ' Avg Fwd Segment Size', 'Subflow Fwd Packets', ' Subflow Fwd Bytes', ' Active Min']
allcols = list(set(ExtraTreeFeatures).union(set(LogisticRegressionFeatures)))
allcols.append(' Label')
numcols = len(allcols)

In [6]:
train_features, train_labels = preprocess2017(riskdata)

NameError: name 'preprocess2017' is not defined

In [5]:
labelset = set(train_labels)
label_dict = {}
i = 0
for l in labelset:
    label_dict[l] = i
    train_labels.replace(to_replace=l, value=i, inplace=True)
    test_labels.replace(to_replace=l, value=i, inplace=True)
    i = i + 1

NameError: name 'train_labels' is not defined